In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()
spark.sql("show catalogs").show()
spark.sql("show databases").show()

+-------------+
|      catalog|
+-------------+
|    hive_prod|
|spark_catalog|
+-------------+

+---------+
|namespace|
+---------+
|  default|
+---------+



In [2]:
spark.sql("CREATE DATABASE IF NOT EXISTS customer_db;").show()

++
||
++
++



In [6]:
spark.sql("USE customer_db");
spark.sql("""
CREATE TABLE customer_db.test002
(
  vendor_id bigint,
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
USING iceberg
PARTITIONED BY (vendor_id)
OPTIONS ('format-version'='2')
;
""").show()

++
||
++
++



In [19]:
spark.sql("show tables").show()

+-----------+---------+-----------+
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|customer_db|  test002|      false|
+-----------+---------+-----------+



In [10]:
spark.sql("""
INSERT INTO customer_db.test002
VALUES (1, 1000371, 1.8, 15.32, 'N'), (2, 1000372, 2.5, 22.15, 'N'), (2, 1000373, 0.9, 9.01, 'N'), (1, 1000374, 8.4, 42.13, 'Y');
""").show()

++
||
++
++



In [11]:
spark.sql("""
SELECT * FROM customer_db.test002
""").show()

+---------+-------+-------------+-----------+------------------+
|vendor_id|trip_id|trip_distance|fare_amount|store_and_fwd_flag|
+---------+-------+-------------+-----------+------------------+
|        1|1000371|          1.8|      15.32|                 N|
|        1|1000374|          8.4|      42.13|                 Y|
|        2|1000372|          2.5|      22.15|                 N|
|        2|1000373|          0.9|       9.01|                 N|
+---------+-------+-------------+-----------+------------------+



In [17]:
spark.sql("""
SELECT * FROM customer_db.test002.files
""").show()

+-------+--------------------+-----------+-------+---------+------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+--------------+---------------------+--------------------+
|content|           file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|        column_sizes|        value_counts|   null_value_counts|nan_value_counts|        lower_bounds|        upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|referenced_data_file|content_offset|content_size_in_bytes|    readable_metrics|
+-------+--------------------+-----------+-------+---------+------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+--------------+

In [18]:
spark.sql("""
SELECT * FROM customer_db.test002.history
""").show()

+--------------------+------------------+---------+-------------------+
|     made_current_at|       snapshot_id|parent_id|is_current_ancestor|
+--------------------+------------------+---------+-------------------+
|2025-04-13 02:12:...|872830672140336580|     NULL|               true|
+--------------------+------------------+---------+-------------------+

